In [ ]:
import copy
import json 

from distutils import dir_util
from dask.distributed import Client, LocalCluster
from pathlib import Path
from urllib.request import urlretrieve

from lc_macro_pipeline.retiler import Retiler
from laserchicken.io.load import load
from laserchicken import filter
from lc_macro_pipeline.data_processing import DataProcessing
from lc_macro_pipeline.geotiff_writer import Geotiff_writer
from lc_macro_pipeline.macro_pipeline import MacroPipeline
from lc_macro_pipeline.classification import Classification

# Macro-ecology LiDAR point-cloud processing pipeline 

## 0. Data Retrieval  and Cluster Setup

Files produced by the pipeline will be saved in the `temp_folder` directory. 

In [ ]:
temp_folder = Path('/var/tmp')

We start by checking whether the test data set is available locally, we otherwise retrieve it from the AHN3 repository.

In [ ]:
testdata_files = ['C_41CZ2.LAZ']

file_paths = [temp_folder.joinpath(f) for f in testdata_files]

for file_path in file_paths:
    if not file_path.is_file():
        file_url = '/'.join(['https://geodata.nationaalgeoregister.nl/ahn3/extract/ahn3_laz', 
                             file_path.name])
        urlretrieve(file_url, file_path)

We then setup the cluster that we will use for the computation using `dask`. For this example, the cluster consists of 3 processes. Note: it is important that single-threaded workers are employed for the tasks that require `laserchicken`!  

In [ ]:
cluster = LocalCluster(processes=True, 
                       n_workers=2, 
                       threads_per_worker=1, 
                       local_directory=temp_folder.joinpath('dask-worker-space'))

## 1. Retiling

The first step in the pipeline is to retile the retrieved point-cloud files to a regular grid, splitting the original data into smaller chuncks that are easier to handle for data processing. The boundaries of the grid and the number of tiles along each axis are set to:

In [ ]:
grid = {
    'min_x': -113107.8100,
    'max_x': 398892.1900,
    'min_y': 214783.8700,
    'max_y': 726783.87,
    'n_tiles_side': 256
}

The retiling of multiple input files consists of independent tasks, which are thus efficiently parallelized. The input controlling all the steps of the retiling is organized in a dictionary.

In [ ]:
retiling_macro = MacroPipeline()

for file_path in file_paths:
    print(file_path)
    
    retiler = Retiler()
    
    retiler.input = {
        'localfs': {
            'input_folder': file_path.parent.as_posix(),
            'input_file': file_path.name,
            'output_folder': temp_folder
        },
        'tiling': grid,
        'split_and_redistribute': {},
        'validate': {}
    }
    
    retiling_macro.add_task(retiler)

retiling_macro.setup_client(cluster=cluster)
res = retiling_macro.run()

## 2. Feature Extraction

Once the files are splitted into tiles of a manageable size, we proceed to the feature extraction stage, which is performed using `laserchicken`. We choose the following two example features:

In [ ]:
feature_names = ["mean_normalized_height", "std_normalized_height"]

The base input dictionary for this step looks like:

In [ ]:
dp_input = {
    "normalize": {
        "cell_size": 1
    },
    "generate_targets": {
        'min_x': -113107.8100,
        'max_x': 398892.1900,
        'min_y': 214783.8700,
        'max_y': 726783.87,
        'n_tiles_side': 256,
        "tile_mesh_size" : 10.0,
        "validate" : True,
    },
    "extract_features": {
        "feature_names": feature_names,
        "volume_type": "cell",
        "volume_size": 10
    }
}

The tiles to which the original input file has been retiled are listed in a record file located in the temporary directory:

In [ ]:
tiles = []
for file_path in file_paths:
    record_file = '_'.join([file_path.stem, 'retile_record.js'])
    with Path(temp_folder/record_file).open() as f:
        record = json.load(f)
    assert record['validated']
    tiles += [Path(temp_folder/tile)
              for tile in record['redistributed_to']]
print([t.as_posix() for t in tiles])

Each tile can be processed independently, so that again one can run the tasks in a parallel fashion.

In [ ]:
dp_macro = MacroPipeline()

for tile in tiles:
    print(tile)
    
    dp = DataProcessing()

    # add tile-specific input to the dictionary
    dp_input['localfs'] = {'input_folder': tile.as_posix(), 'output_folder': tile.parent.as_posix()}
    dp_input['load'] = {}
    dp_input['export_targets'] = {'filename': tile.with_suffix('.ply').name, 'overwrite': True}
    dp_input['generate_targets']['index_tile_x'] = int(tile.name.split('_')[1]) 
    dp_input['generate_targets']['index_tile_y'] = int(tile.name.split('_')[2])
    dp_input['log_config'] = {'filename': '{}_data_processing.log'.format(tile.name)}
    dp.input = copy.deepcopy(dp_input)
    
    dp_macro.add_task(dp)
    
dp_macro.setup_client(cluster=cluster)
res = dp_macro.run()

## 3. Classification of target pointset (optional)
We can classify the target point cloud according to their groud type, based on given cadaster data. 
To mark the types of the points in the point cloud, we can add a new column `ground_type` to the target point cloud. We can use the class code of TOP10NL as the identifier. 

0. Unclassified
1. Gebouw
2. Inrichtingselement
3. Terrein (Polygon)
4. Spoorbaandeel
5. Waterdeel
6. GeografischGebied (Point)
7. FunctioneelGebied
8. Plaats
9. RegistratiefGebied
10. Hoogte
11. Relief (Line String)
12. Wegdeel

Here we present an example of classifying points falls on waterbody with given shp files of waterbody polygon. Stored in `./testdata/shp/`.

First we copy the shp file from `./testdata/shp/` in this repository to `/var/tmp/shp/`:

In [ ]:
dir_util.copy_tree('./testdata/shp/', (temp_folder/'shp').as_posix())

We will classify the target points in `tile_170_107.ply` accoding to shape files stored in `/var/tmp/shp/`.

The pipeline we automatically find out the relavant shp file (`41W_waterdeel.shp` in this case).

We will add a new column `ground_type`, and mark all points which fall in the waterbody polygon `5`, which means `waterdeel`.

We set up the input for classification pipeline as follow:


In [ ]:
classification_input = {
    "localfs":{
        "input_folder": temp_folder.as_posix(),
        "output_folder":(temp_folder/"classified_target_point").as_posix()
    },
    "locate_shp":{
        "shp_dir":"shp"
    },
    "classification":{
        "ground_type":5
    },
  "export_point_cloud":{
    "overwrite":True
  }
}

To accelerate the processing, we only apply classification to the points with `mean_normalized_height` larger than 0.2

Then we excute the pipeline:

In [ ]:
target_tiles = ['tile_170_107.ply', 'tile_170_108.ply']

cl_macro = MacroPipeline()

for tile in target_tiles:
    print(tile)
    
    # Load target point
    f_pc = classification_input['localfs']['input_folder']+'/'+tile
    pc = load(f_pc)
    pc = filter.select_above(pc, 'mean_normalized_height', 0.2) # To accelerate, only use points with mean_normalized_height>0.2

    cl = Classification()
    # add tile-specific input
    classification_input['locate_shp']['point_cloud'] = pc
    classification_input['export_point_cloud']['filename'] = Path(tile).stem+'_classification.ply' 
    cl.input = copy.deepcopy(classification_input)
    
    cl_macro.add_task(cl)

cl_macro.setup_client(cluster=cluster)
res = cl_macro.run()

## 4. GeoTIFF Export

The last step of the pipeline is the transformation of the features extracted from the point-cloud data and 'rasterized' in the target grid to a GeoTIFF file. In this case, the construction of the geotiffs (one per feature) can be performed in parallel: 

In [ ]:
geotiff_macro = MacroPipeline()

for feature_name in feature_names:
    print(feature_name)
    
    gw = Geotiff_writer()
    gw.input = {
        "log_config": {'filename': '{}_geotiff_writer.log'.format(feature_name)},
        "localfs": {'input_folder': temp_folder.as_posix(), 'output_folder': (temp_folder/'geotiff').as_posix()},
        "parse_point_cloud" : {},
        "data_split": {"xSub": 1, "ySub": 1},
        "create_subregion_geotiffs": {
            "outputhandle": "geotiff",
            "band_export": [feature_name]
        }
    }
    
    geotiff_macro.add_task(gw)

geotiff_macro.setup_client(cluster=cluster)
res = geotiff_macro.run()

Finally, we stop the client and the scheduler of the cluster.

In [ ]:
cluster.close()